In [165]:
import os
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot
from scipy import optimize
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
%matplotlib inline

### Tratado de los datos 

In [166]:
def tratadoDatos(dataset,modificarColumnas,eliminarColumnas):
    data =pd.read_csv(dataset,encoding='latin-1')
    columna = data.columns
    for index in modificarColumnas:
        name = columna[index]
        value = list(data[name].unique())
        data[name] = data[name].map(dict(zip(value,[i for i in range(len(value))])))
    #print(data.head())
    data = data.drop([columna[index]for index in eliminarColumnas], axis=1)
    valor_media=data.select_dtypes(include=['number']).mean()
    data = data.fillna(valor_media.round(2))
    return data

In [167]:
def DatosSinteticos(data,listaData,n_filas,clases):
    nuevos_datos=np.empty([n_filas,len(data.columns)])
    ColmunaY = np.random.randint(0,clases, n_filas)
    #print(ColmunaY)
    for i in range(len(data.columns)-1):
        for j in range(0,n_filas):
            pos=ColmunaY[j]
            columna_i =listaData[pos].iloc[:,i]
            muestras_i=np.random.choice(columna_i,1,replace=True)

            pro_cambio = random.random()
            decision = random.randint(0, 1)
            if pro_cambio < 0.2:
               muestras_i = muestras_i + pro_cambio if decision == 0 else muestras_i - pro_cambio

            if(i == 0):
                muestras_i=muestras_i.round(5)
            else:
                muestras_i=muestras_i.round(2)
            #np.random.seed()
            nuevos_datos[j:,i]=muestras_i
            nuevos_datos[j:,len(data.columns)-1]=ColmunaY[j]
    columnas =list(data.columns)
    print(nuevos_datos)
    nuevos_datos_data=pd.DataFrame(nuevos_datos,columns=columnas)
    dataExtend =pd.concat([data,nuevos_datos_data],axis=0)
    #print(dataExtend.tail())
    return dataExtend

In [168]:
def listaDataframes(data,clases,columna_y):
    ListaData=[]
    for i in range(0,clases):
        ListaData.append(data[data[columna_y] == i])
    return ListaData

In [169]:
def col_categoricas(ruta):
    data =pd.read_csv(ruta)
    columnas_numericas = data.select_dtypes(include=['object'])
    col_numericas = np.array([data.columns.get_loc(col) for col in columnas_numericas.columns])
    return col_numericas

In [170]:
def num_clasificaciones(data,columnay):
    value = list(data[columnay].unique())
    print("clases :",value)
    print('Cantidad:',len(value))
    data.info()

In [171]:
def reorganizarDataset(data, posicion_columna):
    # Obtener el nombre de la columna a mover
    nombre_columna = data.columns[posicion_columna]

    # Crear una lista con las columnas en el nuevo orden
    columnas_nuevas = list(data.columns[:posicion_columna]) + list(data.columns[posicion_columna+1:]) + [nombre_columna]
    
    # Crear un nuevo DataFrame con las columnas reorganizadas
    data_reorganizado = data[columnas_nuevas]
    
    return data_reorganizado

### Algortimo one vs all

In [172]:
# valores de prueba para los parámetros theta
theta_t = np.array([-2, -1, 1, 2], dtype=float)

# valores de prueba para las entradas
X_t = np.concatenate([np.ones((5, 1)), np.arange(1, 16).reshape(5, 3, order='F')/10.0], axis=1)

# valores de testeo para las etiquetas
y_t = np.array([1, 0, 1, 0, 1])

# valores de testeo para el parametro de regularizacion
lambda_t = 3

In [173]:
def sigmoid(z):
    """
    Calcula la sigmoide de z.
    """
    return 1.0 / (1.0 + np.exp(-z))

In [174]:
def lrCostFunction(theta, X, y, lambda_):
 
#     alpha = 0.003
#     theta = theta.copy()
    # Inicializa algunos valores utiles
    m = y.size
    epsilon = 1e-8  #evitar diviciones entre 0
    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
#     J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    J = (1 / m) * np.sum(-y.dot(np.log(h+epsilon)) - (1 - y).dot(np.log(1 - h+epsilon))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
#     theta = theta - (alpha / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad
#    return J, theta

In [175]:
def oneVsAll(X, y, num_labels, lambda_,maxiter):
    # algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': maxiter}
        res = optimize.minimize(lrCostFunction, 
                                initial_theta, 
                                (X, (y == c), lambda_), 
                                jac=True, 
                                method='CG',
                                options=options) 
        
        all_theta[c] = res.x

    return all_theta

In [176]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    p = np.zeros(m)
    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
    return p

### Preparar Dataset 

In [177]:
data=pd.read_csv('./Data/Train.csv')
num_clasificaciones(data,'Segmentation')

clases : ['D', 'A', 'B', 'C']
Cantidad: 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [178]:
#data = data.fillna(data.mean)
#data['salario'] = data['salario'].fillna(100) #esto es para rellenar valores nulos en una columna especifica
rutadatset='./Data/Train.csv'
col_categoricas(rutadatset)
data=tratadoDatos(rutadatset,[ 1,  2,  4,  5,  7,  9, 10],[0])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   int64  
 1   Ever_Married     8068 non-null   int64  
 2   Age              8068 non-null   int64  
 3   Graduated        8068 non-null   int64  
 4   Profession       8068 non-null   int64  
 5   Work_Experience  8068 non-null   float64
 6   Spending_Score   8068 non-null   int64  
 7   Family_Size      8068 non-null   float64
 8   Var_1            8068 non-null   int64  
 9   Segmentation     8068 non-null   int64  
dtypes: float64(2), int64(8)
memory usage: 630.4 KB


In [179]:
data.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,0,0,22,0,0,1.00,0,4.0,0,0
1,1,1,38,1,1,2.64,1,3.0,0,1
2,1,1,67,1,1,1.00,0,1.0,1,2
3,0,1,67,1,2,0.00,2,2.0,1,2
4,1,1,40,1,3,2.64,2,6.0,1,1


##### reoriganizar y crear datos sinteticos si lo requiere

In [180]:
#num_clasificaciones(data,'Segmentation')
#Reorganizar las columnas
#datar=reorganizarDataset(data,8)
#datar.head()
Listadata=listaDataframes(data,4,'Segmentation')
data=DatosSinteticos(data,Listadata,5000,4)
#generar un nuevo archivo cvs del nuevo dataset con datos sinteticos
#data.to_csv('TrainSinteticos.csv',index=False)

[[ 0.    1.   39.98 ...  2.    0.    2.  ]
 [ 1.    1.   46.   ...  3.    1.    3.  ]
 [ 0.    1.   53.   ...  1.    0.    1.  ]
 ...
 [ 1.    1.   53.   ...  4.    3.    3.  ]
 [ 1.    1.   18.   ...  3.    4.01  0.  ]
 [ 1.    1.   51.97 ...  2.    1.    2.  ]]


### segmentacion del DataFrame para entrenamiento y Prueba

In [181]:
#separar columna objetivo del dataFrame para X  y asignar objetivo a Y
def segmantacion(data,columnay):
    X = data.drop(columnay, axis=1)
    y = data[columnay]
    X_entre, X_prueba, y_entre, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_entre.to_numpy(), y_entre.to_numpy(),X_prueba.to_numpy(),y_prueba.to_numpy()

In [182]:
X_entre,y_entre,X_prueba,y_prueba=segmantacion(data,'Segmentation')
num_labels = 4
print('Entrenamiento:',X_entre.shape,y_entre.shape,'Prueba:',X_prueba.shape,y_prueba.shape)

Entrenamiento: (10454, 9) (10454,) Prueba: (2614, 9) (2614,)


### Ejecutar algoritmo One vs All

In [183]:
lambda_ = 0.003 #mejor lambda es 0.009 maxiter=500
all_theta = oneVsAll(X_entre, y_entre, num_labels, lambda_,800)
print(all_theta.shape)
print(all_theta)

(4, 10)
[[ 2.26523777e+00 -3.13124578e-01 -7.56628959e-01 -4.25158873e-02
  -1.15206606e+00 -1.23025064e-01  3.02819071e-02 -6.82862637e-01
   8.32417010e-02 -2.80843671e-02]
 [-8.18113300e-01  7.34396757e-02  6.63306876e-02  5.73494443e-03
  -8.88365590e-02  4.00759390e-02  2.67311916e-02 -3.00315582e-01
  -2.29390638e-01 -1.53507972e-03]
 [-2.37496315e+00  6.16889731e-02  3.72494297e-01  1.30347601e-02
   3.60959418e-01  4.83080044e-02 -1.71589844e-02  2.01941346e-01
  -4.81309886e-02 -3.14608959e-03]
 [-3.89082545e+00  1.43300192e-01  5.59410316e-01  1.38960699e-02
   1.02229062e+00  7.27701279e-02 -3.54370607e-02  4.78602401e-01
   1.52004020e-01  2.50693122e-02]]


In [184]:
print(X_prueba.shape)
pred = predictOneVsAll(all_theta, X_prueba)
print('Precision del conjuto de entrenamiento: {:.4f}%'.format(np.mean(pred == y_prueba) * 100))
fila=2
Xp = X_prueba[fila:fila+1, :].copy()
print("se espera de clase :",y_prueba[fila])

Xp = np.concatenate([np.ones((1, 1)), Xp], axis=1)
p = np.argmax(sigmoid(Xp.dot(all_theta.T)), axis = 1)
print("La clase es: ", p)

(2614, 9)
Precision del conjuto de entrenamiento: 46.1744%
se espera de clase : 3.0
La clase es:  [1]
